<center><img src=img/MScAI_brand.png width=70%></center>

# Finite State Machines

For the last 30 years or so, machine learning methods have gradually taken over as the best methods for solving AI problems. In the 30 years before that, most approaches were *symbolic* and logical rather than numerical, probabilistic and learning-oriented. We sometimes call those approaches (half-joking) "good old-fashioned AI".

Task | GOFAI | ML-style AI
-----|-------|------------
Games|Minimax (alpha-beta) | Reinforcement learning
Facts | Logical inference | Probabilistic inference
Natural language | Parsing with grammars | Recurrent neural networks
Agent behaviour | **Finite state machines** | Reinforcement learning

A finite state machine is a neat representation for behaviour. It's still used quite a lot in applications:

* Parsing computer language
* Representing agent behaviour in video games
* Elevators? (A typical textbook example!).

One reason why computer scientists like FSMs is that a computer IS an FSM -- an amazingly complex one. We'll study simple ones.

We'll motivate the idea from an unusual angle and then see an example in game AI.

### Dispatch table

Suppose we had some code like this (an artificial example):

In [1]:
def g0(): print("g0")
def g1(): print("g1")
def g2(): print("g2")
    
def f(a, b, c):
    if a and c:
        if b: 
            g0()
        else:
            g2()
    elif c and not a:
        g1()
    elif c or a:
        g2()

It's hard to predict what will happen for any input!

In [2]:
f(0, 1, 1)

g1


There could even be inputs which don't run any `g`:

In [3]:
f(0, 0, 0)

A *dispatch table* is a different representation for `f`:

In [5]:
dt = {
    (1, 0, 1): g0, # a and c
    (1, 1, 1): g0, # a and c
    (0, 0, 1): g1, # c and not a
    (0, 1, 1): g1, # c and not a
    (1, 0, 0): g2, # c or a
    (1, 1, 0): g2, # c or a
}

def f(a, b, c): dt[(a, b, c)]()

In [6]:
f(0, 1, 1)

g1


With this new representation, we can more easily see that some combinations `(a, b, c)` are not accounted for at all, because they're not in the dictionary `dt`. They'll also throw a `KeyError`.

In [10]:
f(0, 0, 0)

KeyError: (0, 0, 0)

We might decide to have a default to deal with that.

In [11]:
def f(a, b, c):
    try:
        dt[(a, b, c)]()
    except KeyError:
        pass # the original f does nothing in this case

Our claim is that this dispatch table representation may be useful to help us write maintainable and understandable code.

### Finite State Machines

A FSM is a generalisation of a dispatch table in two ways.
1. There are several "states". Every state has its own dispatch table.
2. Every time the function runs with some input, it may not only do something by calling a function, it may also *change state*.

It's often useful to think of the states as representing *internal* data, and the inputs as input from the environment.

### Example: an FSM for a ghost in "Pacman"

<center><img src=img/pacman.png width=40%></center><font size=1>From http://mentalfloss.com/article/49068/how-win-pac-man</font>

[Here's a video to remind us of the gameplay.](https://youtu.be/QjFCmHybgwQ?t=24)

<center><img src=img/pacman_ghost_fsm.svg width=40%></center><font size=1>Inspired by Yannakakis and Togelius, <em>Artificial Intelligence and Games</em></font>


This FSM doesn't have any *actions*. It's just a mapping of the form:

$(\mathrm{state}, \mathrm{input}) \rightarrow \mathrm{state}$

The states are in circles: **Seek Pacman**, **Evade Pacman**, **Go to regenerate**. The possible inputs are Eaten by Pacman, Pacman has power pill, Power pill time up, Finished regenerating. 

E.g. when in state **Evade Pacman**, if we receive the *input* Power pill time up, we'll transition to **Seek Pacman**.

In our example, the states are "high-level" states so there would have to be some extra logic in the game.

We also make an important assumption: no unexpected input will ever appear. E.g. we'll never get input Eaten by Pacman while in the **Go to regenerate** state. If we prefer not to make that assumption, we have to add appropriate arrows, one for every possible input on every state.

### Implementation

In the dispatch table, we just had a function `def f(a, b, c): dt[(a, b, c)]()`. If we wanted to change the function, we would just change `dt`.

Similarly, all FSMs share the same *code* implementation, and differ in their *data*.

Here's the data. It uses the same idea as in the dispatch table: a dictionary mapping "inputs" to outputs, but here the "inputs" are composed of the current state and the input from the environment.

In [10]:
SISAs = {
    # (state, input): state
    ("Seek Pacman", "Pacman has power pill"): 
      "Evade Pacman",
    ("Seek Pacman", "Eaten by Pacman"): 
      "Go to regenerate", 
    ("Evade Pacman", "Power pill time up"): 
      "Seek Pacman",
    ("Evade Pacman", "Eaten by Pacman"): 
      "Go to regenerate",
    ("Go to regenerate", "Finished regenerating"): 
      "Seek Pacman"
    }

Every FSM has a start state. Some also have one or more end states: whenever we reach an end state, we stop executing. If there's no end state, as here, we keep running until the user chooses to stop.

In [11]:
start_state = "Seek Pacman"
end_states = set() # empty set, no end states

Here's one way to write the code:

In [1]:
def fsm(SISAs, start_state, end_states, input_symbols):
    state = start_state
    # (current_state, input_symbol) -> next_state
    for input_symbol in input_symbols:
        yield state
        if state in end_states:
            break
        try:
            state = SISAs[state, input_symbol]
        except KeyError:
            pass # stay in same state

Now let's suppose these are the sequence of game events:

In [16]:
inputs = [
    "Pacman has power pill",
    "Power pill time up",
    "Pacman has power pill",
    "Eaten by Pacman",
    "Finished regenerating"
    ]


And here is how we can use it:

In [17]:
for state in fsm(SISAs, start_state, 
                 end_states, inputs):
    print(state)

Seek Pacman
Evade Pacman
Seek Pacman
Evade Pacman
Go to regenerate


In a real application, our `for`-loop could be more complex, with some other function calls and logic.  

### Exercises

In the lab this week we'll see some exercises using FSMs to model a call centre automated voice response system.